In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("erogluegemen/demand-forecasting-dataset")

print("Path to dataset files:", path)

100%|██████████| 3.29M/3.29M [00:00<00:00, 5.13MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/erogluegemen/demand-forecasting-dataset/versions/1


In [ ]:
import os
import pandas as pd
import kagglehub

# Download latest version and define path within this cell
path = kagglehub.dataset_download("erogluegemen/demand-forecasting-dataset")

df = pd.read_csv(os.path.join(path, 'train.csv'))

Using Colab cache for faster access to the 'demand-forecasting-dataset' dataset.


#                                                            DATA PREPROCESSING

In [ ]:
pd.DataFrame(df)

,store,item,sales,day,month,year
0,1,1,13,1,1,2013
1,1,1,11,2,1,2013
2,1,1,14,3,1,2013
3,1,1,13,4,1,2013
4,1,1,10,5,1,2013
...,...,...,...,...,...,...
912995,10,50,63,27,12,2017
912996,10,50,59,28,12,2017
912997,10,50,74,29,12,2017
912998,10,50,62,30,12,2017


In [ ]:
df.columns

Index(['store', 'item', 'sales', 'day', 'month', 'year'], dtype='object')

In [ ]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [ ]:
df.tail()

,date,store,item,sales
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62
912999,2017-12-31,10,50,82


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


In [ ]:
df.dropna().         # Deletes rows that are exactly the same

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [ ]:
df.drop_duplicates().   #Removes rows with missing values (NaN)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


#                                                            #OUTLIERS CHEAKING

In [ ]:
df['sales'].describe().     #lower outliers:
                            #Values < -30 → ❌ none (sales can’t be negative)
                            #Upper outliers:
                            #Values > 130 → ✅ outliers

                            #bcoz vlaue is below 130 there is no outliers in it


,sales
count,913000.000000
mean,52.250287
std,28.801144
min,0.000000
25%,30.000000
50%,47.000000
75%,70.000000
max,231.000000


#                  #CHEAKING WHICH FEATURE HAS CATEGORIAL OR WHICH IS NUMERICAL
#                  #object → categorical ✅
#                  #category → categorical ✅
# INT/FLOAT = NUMERICAL

In [ ]:
df.dtypes


,0
date,object
store,int64
item,int64
sales,int64


# CHEAK UNIQUE VALUE WHICH MAY BE CATEGORICAL LIKE VALUE BETWEEN 1 TO 10 IS CATEGORICAL AND VALUES IS IN THOUSAND IS NUMERICAL

In [1]:
df.nunique()


NameError: name 'df' is not defined

##                   # NOW CONVRERT CATEGORICAL FEATURE INTO NEMERICAL FEATURE
#                      #NOW 1ST CONVERT DATE COLUMN
# Split up the date column into day,month ,year

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df.drop('date', axis=1, inplace=True)




In [ ]:
df[['day', 'month', 'year']].head()


,day,month,year
0,1,1,2013
1,2,1,2013
2,3,1,2013
3,4,1,2013
4,5,1,2013


In [ ]:
df.dtypes


,0
store,int64
item,int64
sales,int64
day,int32
month,int32
year,int32


# now convert "Store" column into "Numerical Feature "

In [ ]:
df['store'].head()


,store
0,0
1,0
2,0
3,0
4,0


In [ ]:
df['store'] = df['store'].astype('category')
df['store'] = df['store'].cat.codes
df['store'].head()


,store
0,0
1,0
2,0
3,0
4,0


# now swperate feature like "Dependent feature" and "Indepedant feature"

In [ ]:
X = df.drop('sales', axis=1)  #dependant variable .......remove sales column from the dataset beacuse we have to predict sale value it ac t as a input value
y = df['sales']             #independant variable


# NOW SPLIT DATA IN TO TRAIN AND TEST

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import kagglehub

# Data Loading (from q6kyQutagPwg)
path = kagglehub.dataset_download("erogluegemen/demand-forecasting-dataset")
df = pd.read_csv(os.path.join(path, 'train.csv'))

# Date Preprocessing (from qdaLFiT_p5ts)
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df.drop('date', axis=1, inplace=True)

# Store Encoding (from IeVWh17Ew6sm)
df['store'] = df['store'].astype('category')
df['store'] = df['store'].cat.codes

X = df.drop('sales', axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

Using Colab cache for faster access to the 'demand-forecasting-dataset' dataset.


RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
y_pred = rf.predict(X_test)


# Evaluate Performance

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

r2, mae


(0.8470880977638314, 8.333750472966248)

In [ ]:
import pandas as pd

feature_importance = pd.Series(
    rf.feature_importances_, index=X.columns
).sort_values(ascending=False)

feature_importance


,0
item,0.561179
store,0.158333
month,0.134259
day,0.091184
year,0.055044
